# Roadmap

This is a demo to show how we generate a dataset with missing values in outcome and how we apply 

* unadjust complete record analysis (CRA), 
* complete record analysis (CRA)
* inverse probabilty weighting (IPW) without cluster effects
* inverse probabilty weighting (IPW) with cluster effects considered
* multilevel multiple imputation (MMI)

to analysis the dataset with missing outcome 

#### Load in the needed libraries

In [1]:
library(lme4)
library(geepack)
library(jomo)

Loading required package: Matrix


# Functions

### 1. Function to catch errors and warns

In [2]:
myTryCatch <- function(expr) {
  # input: a statement
  # output: the results of the statement, the warnings and the errors in this statement
  warn <- err <- NULL
  value <- withCallingHandlers(
    tryCatch(expr, error=function(e) {
      err <<- e
      NULL
    }), warning=function(w) {
      warn <<- w
      invokeRestart("muffleWarning")
    })
  list(value=value, warning=warn, error=err)
}

### 2. Function to calculate the variance based on ICC

In [3]:
missing_icc=function(icc){
  # input: ICC
  # output: the variance associated with this ICC
  pi=3.142
  a=pi^2/(3*(1/icc-1))
  return(a)
}

#### 2.1 Example

In [4]:
missing_icc(0.1) 

[1] 0.3656357

### 3. Expit function

In [5]:
expit=function(x){y=exp(x)/(1+exp(x));return(y)}

#### 3. Example

In [6]:
expit(0); expit(-100); expit(100)

[1] 0.5

[1] 3.720076e-44

[1] 1

### 4. Function to generate the simulation datasets

In [7]:
dategen=function(k,M,mux=0,varx,icc,mud=0,iccm,intercept){
  K=2*k  # total cluster number
  m=rpois(K,M) # cluster sizes
  N=sum(m) # total individual number
  i=rep(rep(c(0,1),each=k),times=m) # 
  cluster=rep(1:K,times=m)
  vard=missing_icc(icc)
  delta=rep(rnorm(K,mud,sqrt(vard)),times=m)
  x=rnorm(N,mux,sqrt(varx))
  p=expit(1+1.36*i+x+delta)
  y=rbinom(N,1,p)
  alpha=rep(rnorm(K,0,sqrt(missing_icc(iccm))),times=m)
  mis=expit(intercept+i+x+alpha)
  r=rbinom(N,1,mis)
  res=data.frame(y=y,arm=i,x=x,cluster=cluster,delta=delta,mis=mis,r=r)
  return(res)
}

#### 4.1 Example

In [8]:
example=dategen(k=50, M=50, varx=0.2, icc=0.1, iccm=0, intercept=-1.5)
head(example)

y,arm,x,cluster,delta,mis,r
1,0,-0.1469305,1,0.2869029,0.1615242,0
1,0,-0.3741830,1,0.2869029,0.1330585,0
0,0,0.3229701,1,0.2869029,0.2355867,0
1,0,0.6460363,1,0.2869029,0.2986020,1
1,0,-0.1125027,1,0.2869029,0.1662414,0
1,0,0.1697805,1,0.2869029,0.2091231,0


In [9]:
dim(example)

[1] 5081    7

In [10]:
table(example$arm)


   0    1 
2622 2459 

In [11]:
mean(table(example$cluster))

[1] 50.81

### 5. Function to calculate the missingness percentage in dataset

In [12]:
missing_per=function(data){
  # input:
  # output:
  res=sum(data$r)/dim(data)[1]
  return(res)
}

#### 5.1 Example

In [13]:
missing_per(example)

[1] 0.2804566

### 6.Pool function for multiple imputation (MI)

In [14]:
mypool=function(mean0,sd0,num=5,print='no'){
  m=mean(mean0,na.rm=TRUE)
  v=mean(sd0,na.rm=TRUE)
  B=sd(mean0,na.rm=TRUE)
  v_hat=v+(1+1/num)*B
  l=m-1.96*v_hat
  u=m+1.96*v_hat
  if(print=='no'){
    return(list(mean=m,std=v_hat))
  }
  if(print=='yes'){
    print('mean (95% CI)')
    print(paste(round(m,2)," (",round(l,2),',',round(u,2),')',sep=''))
    return(list(mean=m,std=v_hat))
  }
}

# Simulate one dataset with missing outcomes

Since in our simulation, we want to keep each dataset has the same percent of missing in outcome, which is 30%, we need to tune the intercepts when the parameters change in the dataset generation process. 

Here is how we tune the intercept in each round of simulation:

* Given the parameters in the dataset, we get a large range of intercepts from -5 to 5, with step=0.1. Then 100 datasets can be generated. The one with the most closet percent of missingness to 0.3 will be selected

#### Example

In [15]:
k=50;M=50;varx=0.2;icc=0.1;iccm=0.1

mis=c()
for(intercept in seq(-5,5,0.1)){
    temp=dategen(k,M,varx=varx,icc=icc,iccm=iccm,intercept=intercept)  
    mis=c(mis,missing_per(temp))
}
intercept=seq(-5,5,0.1)[which.min(abs(mis-0.3))]
intercept

[1] -1.5

In [16]:
k=50;M=50;varx=0.2;icc=0.1;iccm=0.1
intercept=-1.4
example2=dategen(k,M,varx=varx,icc=icc,iccm=iccm,intercept=intercept)  
missing_per(example2)

[1] 0.3076304

### Analyze the dataset with missing outcomes

Asign the dataset to **d1, d2,** and **d3** to make the variable name consist to the simulation codes

* d1 is the original dataset without missingness
* d3 is the dataset with missing values
* d2 is the d1 dataset that delects the missing values in d3

In [17]:
# d1
d1=example2
head(d1)

y,arm,x,cluster,delta,mis,r
1,0,-0.19685678,1,-0.199188,0.1209250,0
0,0,-0.01159787,1,-0.199188,0.1420410,0
1,0,-0.15495173,1,-0.199188,0.1254508,0
1,0,-0.27905741,1,-0.199188,0.1124556,0
1,0,0.43872608,1,-0.199188,0.2061784,0
1,0,0.77151303,1,-0.199188,0.2659381,0


In [18]:
# d3
d3=example2
d3$y=ifelse(d3$r==1,NA,d3$y)
d3$missing=d3$r
head(d3)

y,arm,x,cluster,delta,mis,r,missing
1,0,-0.19685678,1,-0.199188,0.1209250,0,0
0,0,-0.01159787,1,-0.199188,0.1420410,0,0
1,0,-0.15495173,1,-0.199188,0.1254508,0,0
1,0,-0.27905741,1,-0.199188,0.1124556,0,0
1,0,0.43872608,1,-0.199188,0.2061784,0,0
1,0,0.77151303,1,-0.199188,0.2659381,0,0


In [19]:
# d2
d2=na.omit(d3)
head(d2)

y,arm,x,cluster,delta,mis,r,missing
1,0,-0.19685678,1,-0.199188,0.1209250,0,0
0,0,-0.01159787,1,-0.199188,0.1420410,0,0
1,0,-0.15495173,1,-0.199188,0.1254508,0,0
1,0,-0.27905741,1,-0.199188,0.1124556,0,0
1,0,0.43872608,1,-0.199188,0.2061784,0,0
1,0,0.77151303,1,-0.199188,0.2659381,0,0


### 1.1 True effect with independent working correlation matrix

In [20]:
trues_ind=geeglm(formula=y~arm,id=cluster, data = d1,
                                    family =  binomial("logit"),
                                    corstr = "independence")

In [21]:
summary(trues_ind)$coefficients

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),0.9447017,0.07848507,144.88230,0
arm,1.1779649,0.13776752,73.10903,0


### 1.2 True effect with exchangeable working correlation matrix

In [22]:
trues_ex=geeglm(formula=y~arm,id=cluster, data = d1,
                                   family =  binomial("logit"),
                                   corstr = "exchangeable")

In [23]:
summary(trues_ex)$coefficients

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),0.9393288,0.07764006,146.37387,0
arm,1.1951098,0.13400613,79.53647,0


### 2.1 Unadjusted CRA with independent working correlation matrix

In [24]:
d2=na.omit(d3)
ucra_ind=geeglm(formula=y~arm,id=cluster, data = d2,
                                   family =  binomial("logit"),
                                   corstr = "independence")

In [25]:
summary(ucra_ind)$coefficients

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),0.9322216,0.07642514,148.78727,0.000000e+00
arm,1.1109600,0.14831494,56.10829,6.861178e-14


### 2.2 Unadjusted CRA with exchangeable working correlation matrix

In [26]:
ucra_ex=geeglm(formula=y~arm,id=cluster, data = d2,
                                  family =  binomial("logit"),
                                  corstr = "exchangeable")

In [27]:
summary(ucra_ex)$coefficients

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),0.9270717,0.0743951,155.28802,0.000000e+00
arm,1.0907114,0.1440184,57.35668,3.630429e-14


### 3.1 Adjusted CRA with independent working correlation matrix

In [28]:
cra_ind=geeglm(formula=y~x+arm,id=cluster, data = d2,
                                  family =  binomial("logit"),
                                  corstr = "independence")

In [29]:
summary(cra_ind)$coefficients

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),0.9924606,0.08088748,150.54420,0.000000e+00
x,0.8655646,0.08620781,100.81048,0.000000e+00
arm,1.1665538,0.14921610,61.11928,5.329071e-15


### 3.2 Adjusted CRA with exchangeable working correlation matrix

In [30]:
cra_ex=geeglm(formula=y~x+arm,id=cluster, data = d2,
                                 family =  binomial("logit"),
                                 corstr = "exchangeable")

In [31]:
summary(cra_ex)$coefficients

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),0.9890455,0.07872937,157.81890,0.000000e+00
x,0.8702397,0.08514418,104.46430,0.000000e+00
arm,1.1409707,0.14678714,60.41893,7.660539e-15


### 4.1 IPW, without consideration of cluster effects, independent working correlation matrix

By preforming IPW, we firstly calculate the weights of missingness for each participant. 

* w1: the weights without consideration of cluster effects
* w2: the weights with consideration of cluster effects 

In [32]:
w1=glm(missing ~x + arm, data = d3, family = binomial(link='logit'))
w2=glmer(missing ~x + arm+(1|cluster) , data = d3,family = binomial(link='logit'))
summary(w1)$coefficients

,Estimate,Std. Error,z value,Pr(>|z|)
(Intercept),-1.3237042,0.04980259,-26.57902,1.186680e-155
x,0.8845521,0.07402777,11.94892,6.577247e-33
arm,0.8928844,0.06473582,13.79274,2.818423e-43


In [33]:
summary(w2)$coefficients

,Estimate,Std. Error,z value,Pr(>|z|)
(Intercept),-1.4240443,0.10602023,-13.43182,3.936319e-41
x,0.9399899,0.07755602,12.12014,8.261648e-34
arm,0.9331475,0.14664129,6.36347,1.972453e-10


In [34]:
w1=predict(w1,type="response")
w2=predict(w2,type="response") 
# add the weights in the dataset
d3$weight=1/w1
d3$weight2=1/w2

In [35]:
head(d3)

y,arm,x,cluster,delta,mis,r,missing,weight,weight2
1,0,-0.19685678,1,-0.199188,0.1209250,0,0,5.471993,7.925057
0,0,-0.01159787,1,-0.199188,0.1420410,0,0,4.796058,6.818280
1,0,-0.15495173,1,-0.199188,0.1254508,0,0,5.309264,7.657579
1,0,-0.27905741,1,-0.199188,0.1124556,0,0,5.809268,8.481356
1,0,0.43872608,1,-0.199188,0.2061784,0,0,3.548814,4.810288
1,0,0.77151303,1,-0.199188,0.2659381,0,0,2.898873,3.786785


Use IPW to analyze the data

In [36]:
ipw_ind=geeglm(formula=y~arm,id=cluster, data = d3,
                                  family =  binomial("logit"),
                                  weights = d3$weight,
                                  corstr = "independence")

Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”

In [37]:
summary(ipw_ind)$coefficients

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),0.8245646,0.07535509,119.73584,0.000000e+00
arm,1.0997879,0.15196915,52.37295,4.589662e-13


### 4.2 IPW, without consideration of cluster effects, exchangeable working correlation matrix

In [38]:
ipw_ex=geeglm(formula=y~arm,id=cluster, data = d3,
                                 family =  binomial("logit"),
                                 weights = d3$weight,
                                 corstr = "exchangeable")

Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”

In [39]:
summary(ipw_ex)$coefficients

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),0.7466171,0.07512865,98.76083,0.000000e+00
arm,1.0940630,0.15100518,52.49295,4.317657e-13


### 5.1 IPW, with consideration of cluster effects, independent working correlation matrix

In [40]:
ipw_clu_ind=geeglm(formula=y~arm,id=cluster, data = d3,
                                      family =  binomial("logit"),
                                      weights = d3$weight2,
                                      corstr = "independence")

Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”

In [41]:
summary(ipw_clu_ind)$coefficients

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),0.8290968,0.08345892,98.68814,0.000000e+00
arm,1.2379548,0.16640605,55.34410,1.011413e-13


### 5.2 IPW, with consideration of cluster effects, exchangeable working correlation matrix

In [42]:
ipw_clu_ex=geeglm(formula=y~arm,id=cluster, data = d3,
                                     family =  binomial("logit"),
                                     weights = d3$weight2,
                                     corstr = "exchangeable")

Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”

In [43]:
summary(ipw_clu_ex)$coefficients

,Estimate,Std.err,Wald,Pr(>|W|)
(Intercept),0.7240982,0.08036469,81.18287,0.000000e+00
arm,1.2548667,0.16595839,57.17376,3.985701e-14


### 6 Multiilevel multiple imputation

#### Step 1: Imputate the dataset for 15 times. The new dataset is called as **imp**

In [44]:
Nimp=15 # imputation time
data.miss=d3
y.cat= data.frame(outcome=data.miss$y)  # data frame for response variables with missing values
y.numcat=c(2)                                 # number of levels in outcome variable
clus=data.frame(clus=data.miss$cluster)          # data frame for clusters
nobs=dim(data.miss)[1]

x= data.frame(intercept=rep(1,nobs),covariate=data.miss$x,group=data.miss$arm)
imp = jomo1rancat(Y.cat=y.cat, Y.numcat=y.numcat, X=x,
        clus=clus,nburn=100, nbetween=25, nimp=Nimp,output=0)

In [45]:
dim(d3);dim(imp)

[1] 4967   10

[1] 79472     8

In [46]:
dim(d3)[1]*(1+15)

[1] 79472

#### Step 2: Analyze the 15 imputed datasets separately and save their results

In [47]:
mmi_est_ind=c();mmi_std_ind=c();mmi_warn_ind=c()
mmi_est_ex=c();mmi_std_ex=c();mmi_warn_ex=c()

for(i in 1:Nimp){
        temp=imp[imp$Imputation==i,]
        rownames(temp)=NULL
        temp$outcome=as.numeric(temp$outcome)-1
        
        mmi_ind=geeglm(formula=outcome~group,
                                  id=clus , data = temp,
                                  family =  binomial("logit"),
                                  corstr = "independence")
        
        mmi_ex=geeglm(formula=outcome~group,
                                 id=clus , data = temp,
                                 family =  binomial("logit"),
                                 corstr = "exchangeable")
    
       t1=summary(mmi_ind)$coefficients['group','Estimate']
       t2=summary(mmi_ind)$coefficients['group','Std.err']
       mmi_est_ind=c(mmi_est_ind,t1)
       mmi_std_ind=c(mmi_std_ind,t2)
    
       t1=summary(mmi_ex)$coefficients['group','Estimate']
       t2=summary(mmi_ex)$coefficients['group','Std.err']
       mmi_est_ex=c(mmi_est_ex,t1)
       mmi_std_ex=c(mmi_std_ex,t2)
}

#### Results

In [48]:
mmi_est_ind

[1] 1.0941299 1.0902067 1.1490709 1.1002326 1.0449205 1.0739801 0.9986882
 [8] 1.1042810 1.1563770 1.1986535 0.9673196 1.1833756 1.1083384 1.1333784
[15] 1.0471177

In [49]:
mmi_std_ind

[1] 0.1335618 0.1384778 0.1386663 0.1394308 0.1387710 0.1442096 0.1367816
 [8] 0.1438383 0.1360366 0.1326949 0.1327264 0.1243302 0.1212095 0.1342285
[15] 0.1236102

In [50]:
mmi_est_ex

[1] 1.101328 1.111150 1.165737 1.114563 1.063568 1.097548 1.017596 1.120879
 [9] 1.167291 1.203787 0.986204 1.196369 1.112415 1.138951 1.060450

In [51]:
mmi_std_ex

[1] 0.1321010 0.1316228 0.1365675 0.1364100 0.1337329 0.1388020 0.1332927
 [8] 0.1411017 0.1303945 0.1312590 0.1295432 0.1218977 0.1200392 0.1322278
[15] 0.1221604

#### Step 3: Pool the saved results

In [52]:
mmi_res1=mypool(mmi_est_ind,mmi_std_ind,num=Nimp)
mmi_res2=mypool(mmi_est_ex,mmi_std_ex,num=Nimp)

#### The final results

In [53]:
mmi_res1  # with independent working correlation matrix

$mean
[1] 1.096671

$std
[1] 0.2031162

In [54]:
mmi_res2 # with exchangeable working correlation matrix

$mean
[1] 1.110522

$std
[1] 0.1966242

# Try to run the whole codes together 

Run times of simulations (Do not set to T=1000 since it takes a lot of time) 

In [55]:
T=5

s1=Sys.time()

k=50; icc=0.1; iccm=0.1

      true_est_ind=c();true_std_ind=c();true_warn_ind=c()
      true_est_ex=c();true_std_ex=c();true_warn_ex=c()
      ucra_est_ind=c();ucra_std_ind=c();ucra_warn_ind=c()
      ucra_est_ex=c();ucra_std_ex=c();ucra_warn_ex=c()
      cra_est_ind=c();cra_std_ind=c();cra_warn_ind=c()
      cra_est_ex=c();cra_std_ex=c();cra_warn_ex=c()
      ipw_est_ind=c();ipw_std_ind=c();ipw_warn_ind=c()
      ipw_est_ex=c();ipw_std_ex=c();ipw_warn_ex=c()
      ipw_clu_est_ind=c();ipw_clu_std_ind=c();ipw_clu_warn_ind=c()
      ipw_clu_est_ex=c();ipw_clu_std_ex=c();ipw_clu_warn_ex=c()
      miss=c()
      maxw1=c(); maxw2=c()
      
      for(times in 1:T){
        print(paste('icc',icc,'iccm',iccm,'times',times))
        set.seed(times)
        
        #tune the intercept
        mis=c()
        for(intercept in seq(-5,5,0.1)){
          temp=dategen(k,M,varx=varx,icc=icc,iccm=iccm,intercept=intercept)  
          mis=c(mis,missing_per(temp))
        }
        intercept=seq(-5,5,0.1)[which.min(abs(mis-0.3))]
        
        #
        d1=dategen(k,M,varx=varx,icc=icc,iccm=iccm,intercept=intercept)
        d3=d1
        d3$y=ifelse(d3$r==1,NA,d3$y)
        d3$missing=d3$r
        d2=na.omit(d3)
        
        #calculate the weights
        w1=glm(missing ~x + arm, data = d3,
               family = binomial(link='logit'))
        w2=glmer(missing ~x + arm+(1|cluster) , data = d3,
                 family = binomial(link='logit'))
        
        w1=predict(w1,type="response")
        w2=predict(w2,type="response")
        
        d3$weight=1/w1
        d3$weight2=1/w2
        
        maxw1=c(maxw1,max(d3$weight))
        maxw2=c(maxw2,max(d3$weight2))
        miss=c(miss,sum(d1$r))
        
        ### True effect
        trues_ind=myTryCatch(geeglm(formula=y~arm,id=cluster, data = d1,
                                    family =  binomial("logit"),
                                    corstr = "independence"))
        trues_ex=myTryCatch(geeglm(formula=y~arm,id=cluster, data = d1,
                                   family =  binomial("logit"),
                                   corstr = "exchangeable"))
        
        ### Unadjusted CRA
        ucra_ind=myTryCatch(geeglm(formula=y~arm,id=cluster, data = d2,
                                   family =  binomial("logit"),
                                   corstr = "independence"))
        ucra_ex=myTryCatch(geeglm(formula=y~arm,id=cluster, data = d2,
                                  family =  binomial("logit"),
                                  corstr = "exchangeable"))
        
        ### Adjusted CRA
        cra_ind=myTryCatch(geeglm(formula=y~x+arm,id=cluster, data = d2,
                                  family =  binomial("logit"),
                                  corstr = "independence"))
        cra_ex=myTryCatch(geeglm(formula=y~x+arm,id=cluster, data = d2,
                                 family =  binomial("logit"),
                                 corstr = "exchangeable"))
        
        ### IPW
        ipw_ind=myTryCatch(geeglm(formula=y~arm,id=cluster, data = d3,
                                  family =  binomial("logit"),
                                  weights = d3$weight,
                                  corstr = "independence"))
        ipw_ex=myTryCatch(geeglm(formula=y~arm,id=cluster, data = d3,
                                 family =  binomial("logit"),
                                 weights = d3$weight,
                                 corstr = "exchangeable"))
        ### IPW cluster
        ipw_clu_ind=myTryCatch(geeglm(formula=y~arm,id=cluster, data = d3,
                                      family =  binomial("logit"),
                                      weights = d3$weight2,
                                      corstr = "independence"))
        ipw_clu_ex=myTryCatch(geeglm(formula=y~arm,id=cluster, data = d3,
                                     family =  binomial("logit"),
                                     weights = d3$weight2,
                                     corstr = "exchangeable"))
        
        #### save results:
        if(is.null(trues_ind$value)==0){
          t1=summary(trues_ind$value)$coefficients['arm','Estimate']
          t2=summary(trues_ind$value)$coefficients['arm','Std.err']
          true_est_ind=c(true_est_ind,t1)
          true_std_ind=c(true_std_ind,t2)
          if(summary(trues_ind$value)$error==1){true_warn_ind=c(true_warn_ind,times)}
          if(summary(trues_ind$value)$error==0){true_warn_ind=c(true_warn_ind,0)}
        }
        if(is.null(trues_ind$value)==1){
          true_est_ind=c(true_est_ind,NA)
          true_std_ind=c(true_std_ind,NA)
          true_warn_ind=c(true_warn_ind,times)
        }
        
        if(is.null(trues_ex$value)==0){
          t1=summary(trues_ex$value)$coefficients['arm','Estimate']
          t2=summary(trues_ex$value)$coefficients['arm','Std.err']
          true_est_ex=c(true_est_ex,t1)
          true_std_ex=c(true_std_ex,t2)
          if(summary(trues_ex$value)$error==1){true_warn_ex=c(true_warn_ex,times)}
          if(summary(trues_ex$value)$error==0){true_warn_ex=c(true_warn_ex,0)}
        }
        if(is.null(trues_ex$value)==1){
          true_est_ex=c(true_est_ex,NA)
          true_std_ex=c(true_std_ex,NA)
          true_warn_ex=c(true_warn_ex,times)
        }
        
        if(is.null(ucra_ind$value)==0){
          t1=summary(ucra_ind$value)$coefficients['arm','Estimate']
          t2=summary(ucra_ind$value)$coefficients['arm','Std.err']
          ucra_est_ind=c(ucra_est_ind,t1)
          ucra_std_ind=c(ucra_std_ind,t2)
          if(summary(ucra_ind$value)$error==1){ucra_warn_ind=c(ucra_warn_ind,times)}
          if(summary(ucra_ind$value)$error==0){ucra_warn_ind=c(ucra_warn_ind,0)}
        }
        if(is.null(ucra_ind$value)==1){
          ucra_est_ind=c(ucra_est_ind,NA)
          ucra_std_ind=c(ucra_std_ind,NA)
          ucra_warn_ind=c(ucra_warn_ind,times)
        }
        
        if(is.null(ucra_ex$value)==0){
          t1=summary(ucra_ex$value)$coefficients['arm','Estimate']
          t2=summary(ucra_ex$value)$coefficients['arm','Std.err']
          ucra_est_ex=c(ucra_est_ex,t1)
          ucra_std_ex=c(ucra_std_ex,t2)
          if(summary(ucra_ex$value)$error==1){ucra_warn_ex=c(ucra_warn_ex,times)}
          if(summary(ucra_ex$value)$error==0){ucra_warn_ex=c(ucra_warn_ex,0)}
        }
        if(is.null(ucra_ex$value)==1){
          ucra_est_ex=c(ucra_est_ex,NA)
          ucra_std_ex=c(ucra_std_ex,NA)
          ucra_warn_ex=c(ucra_warn_ex,times)
        }
        
        if(is.null(cra_ind$value)==0){
          t1=summary(cra_ind$value)$coefficients['arm','Estimate']
          t2=summary(cra_ind$value)$coefficients['arm','Std.err']
          cra_est_ind=c(cra_est_ind,t1)
          cra_std_ind=c(cra_std_ind,t2)
          if(summary(cra_ind$value)$error==1){cra_warn_ind=c(cra_warn_ind,times)}
          if(summary(cra_ind$value)$error==0){cra_warn_ind=c(cra_warn_ind,0)}
        }
        if(is.null(cra_ind$value)==1){
          cra_est_ind=c(cra_est_ind,NA)
          cra_std_ind=c(cra_std_ind,NA)
          cra_warn_ind=c(cra_warn_ind,times)
        }
        
        if(is.null(cra_ex$value)==0){
          t1=summary(cra_ex$value)$coefficients['arm','Estimate']
          t2=summary(cra_ex$value)$coefficients['arm','Std.err']
          cra_est_ex=c(cra_est_ex,t1)
          cra_std_ex=c(cra_std_ex,t2)
          if(summary(cra_ex$value)$error==1){cra_warn_ex=c(cra_warn_ex,times)}
          if(summary(cra_ex$value)$error==0){cra_warn_ex=c(cra_warn_ex,0)}
        }
        if(is.null(cra_ex$value)==1){
          cra_est_ex=c(cra_est_ex,NA)
          cra_std_ex=c(cra_std_ex,NA)
          cra_warn_ex=c(cra_warn_ex,times)
        }
        
        if(is.null(ipw_ind$value)==0){
          t1=summary(ipw_ind$value)$coefficients['arm','Estimate']
          t2=summary(ipw_ind$value)$coefficients['arm','Std.err']
          ipw_est_ind=c(ipw_est_ind,t1)
          ipw_std_ind=c(ipw_std_ind,t2)
          if(summary(ipw_ind$value)$error==1){ipw_warn_ind=c(ipw_warn_ind,times)}
          if(summary(ipw_ind$value)$error==0){ipw_warn_ind=c(ipw_warn_ind,0)}
        }
        if(is.null(ipw_ind$value)==1){
          ipw_est_ind=c(ipw_est_ind,NA)
          ipw_std_ind=c(ipw_std_ind,NA)
          ipw_warn_ind=c(ipw_warn_ind,times)
        }
        
        if(is.null(ipw_ex$value)==0){
          t1=summary(ipw_ex$value)$coefficients['arm','Estimate']
          t2=summary(ipw_ex$value)$coefficients['arm','Std.err']
          ipw_est_ex=c(ipw_est_ex,t1)
          ipw_std_ex=c(ipw_std_ex,t2)
          if(summary(ipw_ex$value)$error==1){ipw_warn_ex=c(ipw_warn_ex,times)}
          if(summary(ipw_ex$value)$error==0){ipw_warn_ex=c(ipw_warn_ex,0)}
        }
        if(is.null(ipw_ex$value)==1){
          ipw_est_ex=c(ipw_est_ex,NA)
          ipw_std_ex=c(ipw_std_ex,NA)
          ipw_warn_ex=c(ipw_warn_ex,times)
        }
        
        if(is.null(ipw_clu_ind$value)==0){
          t1=summary(ipw_clu_ind$value)$coefficients['arm','Estimate']
          t2=summary(ipw_clu_ind$value)$coefficients['arm','Std.err']
          ipw_clu_est_ind=c(ipw_clu_est_ind,t1)
          ipw_clu_std_ind=c(ipw_clu_std_ind,t2)
          if(summary(ipw_clu_ind$value)$error==1){ipw_clu_warn_ind=c(ipw_clu_warn_ind,times)}
          if(summary(ipw_clu_ind$value)$error==0){ipw_clu_warn_ind=c(ipw_clu_warn_ind,0)}
        }
        if(is.null(ipw_clu_ind$value)==1){
          ipw_clu_est_ind=c(ipw_clu_est_ind,NA)
          ipw_clu_std_ind=c(ipw_clu_std_ind,NA)
          ipw_clu_warn_ind=c(ipw_clu_warn_ind,times)
        }
        
        if(is.null(ipw_clu_ex$value)==0){
          t1=summary(ipw_clu_ex$value)$coefficients['arm','Estimate']
          t2=summary(ipw_clu_ex$value)$coefficients['arm','Std.err']
          ipw_clu_est_ex=c(ipw_clu_est_ex,t1)
          ipw_clu_std_ex=c(ipw_clu_std_ex,t2)
          if(summary(ipw_clu_ex$value)$error==1){ipw_clu_warn_ex=c(ipw_clu_warn_ex,times)}
          if(summary(ipw_clu_ex$value)$error==0){ipw_clu_warn_ex=c(ipw_clu_warn_ex,0)}
        }
        if(is.null(ipw_clu_ex$value)==1){
          ipw_clu_est_ex=c(ipw_clu_est_ex,NA)
          ipw_clu_std_ex=c(ipw_clu_std_ex,NA)
          ipw_clu_warn_ex=c(ipw_clu_warn_ex,times)
        }
        
      }
      
      est_ind=data.frame(true_est_ind=true_est_ind,ucra_est_ind=ucra_est_ind,cra_est_ind=cra_est_ind,
                         ipw_est_ind=ipw_est_ind,ipw_clu_est_ind=ipw_clu_est_ind)
      est_ex=data.frame(true_est_ex=true_est_ex,ucra_est_ex=ucra_est_ex,cra_est_ex=cra_est_ex,
                        ipw_est_ex=ipw_est_ex,ipw_clu_est_ex=ipw_clu_est_ex)
      
      std_ind=data.frame(true_std_ind=true_std_ind,ucra_std_ind=ucra_std_ind,cra_std_ind=cra_std_ind,
                         ipw_std_ind=ipw_std_ind,ipw_clu_std_ind=ipw_clu_std_ind)
      std_ex=data.frame(true_std_ex=true_std_ex,ucra_std_ex=ucra_std_ex,cra_std_ex=cra_std_ex,
                        ipw_std_ex=ipw_std_ex,ipw_clu_std_ex=ipw_clu_std_ex)
      
      warn_ind=data.frame(true_warn_ind=true_warn_ind,ucra_warn_ind=ucra_warn_ind,cra_warn_ind=cra_warn_ind,
                          ipw_warn_ind=ipw_warn_ind,ipw_clu_warn_ind=ipw_clu_warn_ind)
      warn_ex=data.frame(true_warn_ex=true_warn_ex,ucra_warn_ex=ucra_warn_ex,cra_warn_ex=cra_warn_ex,
                         ipw_warn_ex=ipw_warn_ex,ipw_clu_warn_ex=ipw_clu_warn_ex)
      
      result=list(est_ind=est_ind,est_ex=est_ex,std_ind=std_ind,
                  std_ex=std_ex,warn_ind=warn_ind,warn_ex=warn_ex,miss=miss,maxw1=maxw1,maxw2=maxw2)
      names=paste('wrapk',k,icc,iccm,'.RData',sep='')
      #save(result,file=names)
      print(result)

e1=Sys.time() 

[1] "icc 0.1 iccm 0.1 times 1"
[1] "icc 0.1 iccm 0.1 times 2"
[1] "icc 0.1 iccm 0.1 times 3"
[1] "icc 0.1 iccm 0.1 times 4"
[1] "icc 0.1 iccm 0.1 times 5"
$est_ind
  true_est_ind ucra_est_ind cra_est_ind ipw_est_ind ipw_clu_est_ind
1     1.310384     1.159828    1.224555    1.210207        1.159971
2     1.363460     1.309820    1.413219    1.344728        1.401885
3     1.271641     1.247465    1.317396    1.355534        1.352649
4     1.288215     1.341463    1.382355    1.370798        1.526852
5     1.397283     1.319086    1.376324    1.417114        1.326664

$est_ex
  true_est_ex ucra_est_ex cra_est_ex ipw_est_ex ipw_clu_est_ex
1    1.315306    1.179598   1.273972   1.281814       1.206704
2    1.354091    1.287376   1.372334   1.376332       1.424973
3    1.257005    1.247020   1.386951   1.469710       1.438932
4    1.283022    1.313598   1.360069   1.435135       1.605032
5    1.415773    1.342463   1.433274   1.549111       1.431345

$std_ind
  true_std_ind ucra_std_ind cra